In [13]:
import pandas as pd
import numpy as np
import time
!pip install influxdb
!pip install requests
from influxdb import DataFrameClient
import requests
from datetime import datetime

In [14]:
# apiData class
class ApiData:
    def __init__(self,date,infected,newInf,tested,newTst,recovered,newRcv,deceased,newDcd):
        self.date = date
        self.infected = infected 
        self.newInf = newInf
        self.tested = tested
        self.newTst = newTst
        self.recovered = recovered
        self.newRcv = newRcv
        self.deceased = deceased
        self.newDcd = newDcd

In [15]:
def init_Database():
    #get data from api
    url = 'https://api.apify.com/v2/datasets/n1XtXTelVG5dJhDhy/items?format=json&clean=1'
    json_data = requests.get(url).json()
    #example dataprint (json_data[0]['lastUpdatedAtApify'][0:10],json_data[0]['infected'],json_data[0]['tested'],json_data[0]['recovered'],json_data[0]['deceased'])
    #data strucure
    data = []
    #put data in data structure
    i=1
    now = datetime.now()
    date= now.strftime("%y-%m-%d")
    #date,infected,newInf,tested,newTst,recovered,newRcv,deceased,newDcd
    try:
        while json_data[i]['lastUpdatedAtApify'][0:10] != now :

                print(json_data[i]['lastUpdatedAtApify'][0:10],json_data[i]['infected'],json_data[i]['infected']-json_data[i-1]['infected'],json_data[i]['tested'],json_data[i]['tested']-json_data[i-1]['tested'],json_data[i]['recovered'],json_data[i]['recovered']-json_data[i-1]['recovered'],json_data[i]['deceased'],json_data[i]['deceased']-json_data[i-1]['deceased'])
                if i == 0 :
                    data.append(ApiData(json_data[i]['lastUpdatedAtApify'][0:10],json_data[i]['infected'],0,json_data[i]['tested'],0,json_data[i]['recovered'],0,json_data[i]['deceased'],0))

                else:
                    data.append(ApiData(json_data[i]['lastUpdatedAtApify'][0:10],json_data[i]['infected'],json_data[i]['infected']-json_data[i-1]['infected'],json_data[i]['tested'],json_data[i]['tested']-json_data[i-1]['tested'],json_data[i]['recovered'],json_data[i]['recovered']-json_data[i-1]['recovered'],json_data[i]['deceased'],json_data[i]['deceased']-json_data[i-1]['deceased']))
                i=i+1
    except:
            print('')
    df = pd.DataFrame(columns=['date','infected','newInfected','tested','newTested','recovered','newRecovered','deceased','newDeceased'])
    print (df)
    for i in range(len(data)):
        df.loc[i] = [data[i].date,data[i].infected,data[i].newInf,data[i].tested,data[i].newTst,data[i].recovered,data[i].newRcv,data[i].deceased,data[i].newDcd]    
    user = 'root'
    password = 'root'
    host='influxdb' #172.28.1.2 #localhost #3.14.5.146 - kozos osztaly InfluxDB szerver
    port=8086
    dbname='covid19Romania' #az en adatbazis nevem
    protocol = 'line' #json
    measurement='teszt'
    client = DataFrameClient(host, port, user, password, dbname)
    #crete database
    client.drop_database(dbname)
    client.drop_retention_policy(dbname)
    client.create_database(dbname)
    client.create_retention_policy(dbname, '1000d', 1, default=True)
    df = pd.DataFrame(df).set_index('date')
    #df = pd.DataFrame(columns=['date','infected','newInfected','tested','newTested','recovered','newRecovered','deceased','newDeceased'])
    df.index = pd.to_datetime(df.index)
    df['infected']=df['infected'].astype(int)
    df['tested']=df['tested'].astype(int)
    df['recovered']=df['recovered'].astype(int)
    df['deceased']=df['deceased'].astype(int)

    df['newInfected']=df['newInfected'].astype(int)
    df['newTested']=df['newTested'].astype(int)
    df['newRecovered']=df['newRecovered'].astype(int)
    df['newDeceased']=df['newDeceased'].astype(int)
    client.write_points(df, 'database', protocol=protocol,
                            field_columns=['infected','newInfected','tested','newTested','recovered','newRecovered','deceased','newDeceased'])
    print('Database updated succesfully')

In [16]:
init_Database()

2020-04-20 8936 190 98491 4880 2017 125 451 17
2020-04-21 9242 306 101552 3061 2153 136 482 31
2020-04-22 9710 468 106357 4805 2406 253 508 26
2020-04-23 10096 386 113336 6979 2478 72 527 19
2020-04-24 10417 321 121602 8266 2817 339 552 25
2020-04-25 10635 218 126645 5043 2890 73 575 23
2020-04-26 11036 401 136518 9873 3054 164 608 33
2020-04-27 11339 303 143834 7316 3141 87 631 23
2020-04-28 11616 277 150309 6475 3404 263 650 19
2020-04-29 11978 362 166993 16684 3569 165 675 25
2020-04-30 12240 262 175374 8381 4017 448 695 20
2020-05-01 12567 327 183688 8314 4328 311 726 31
2020-05-02 12732 165 190540 6852 4547 219 755 29
2020-05-03 13163 431 195508 4968 4869 322 780 25
2020-05-04 13512 349 199068 3560 5269 400 803 23
2020-05-05 13837 325 205842 6774 5454 185 827 24
2020-05-06 14107 270 217139 11297 5788 334 858 31
2020-05-07 14499 392 226613 9474 6144 356 876 18
2020-05-08 14811 312 237280 10667 6423 279 898 22
2020-05-09 15131 320 248056 10776 6912 489 926 28
2020-05-10 15362 231 25